In [ ]:
import json
import pandas as pd
import requests
import numpy as np
import sqlalchemy
from datetime import datetime, timedelta, timezone

def lambda_handler(event, context):
    # connect to database
    schema="the name of your schema"
    host="provide the aws host"
    user="admin"
    password="your password for mySQL"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    # execute the script to update the df
    get_arrivles_berlin(con)
    
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


def get_arrivles_berlin(con):
    
    # Get tomorrow's date
    tomorrow_time_date = datetime.now() + timedelta(days=1)
    tomorrow = tomorrow_time_date.strftime('%Y-%m-%d')
    
    # Get the flights arriving between 00:00 - 12:00 o'clock
    API_key = "******************************************"
    
    airport_icoa = "EDDB"
    to_local_time = f"{tomorrow}T00:00"
    from_local_time = f"{tomorrow}T12:00"
    
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"
    
    querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
    
    headers = {
    	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
    	"X-RapidAPI-Key": API_key
    }
    
    response_am = requests.request("GET", url, headers=headers, params=querystring)
    response_json_am = response_am.json()
    
    # Get the flights arriving between 12:00 - 23:59 o'clock
    API_key = "******************************************"
    
    airport_icoa = "EDDB"
    to_local_time = f"{tomorrow}T12:00"
    from_local_time = f"{tomorrow}T23:59"
    
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"
    
    querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
    
    headers = {
    	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
    	"X-RapidAPI-Key": API_key
    }

    response_pm = requests.request("GET", url, headers=headers, params=querystring)
    response_json_pm = response_pm.json()
    
    
    # Transform the data from json format to a pd data frame
    arrivals_first_half = pd.json_normalize(response_am.json()["arrivals"])
    arrivals_second_half = pd.json_normalize(response_pm.json()["arrivals"])
    
    # select the columns you want to include into the database for the am flights
    arrivals_first_half = (
        arrivals_first_half
        .filter(['departure.airport.name','arrival.scheduledTimeLocal',
        'arrival.terminal','status','aircraft.model'])
        .assign(icao_arrival_airport = airport_icoa)
    )
    
    # select the columns you want to include into the database for the pm flights
    arrivals_second_half = (
        arrivals_second_half
        .filter(['departure.airport.name','arrival.scheduledTimeLocal',
        'arrival.terminal','status','aircraft.model'])
        .assign(icao_arrival_airport = airport_icoa)
    )
    
    # Append the half day data frames into one df which covers the whole day
    df_ber_flights_arrivals = arrivals_first_half.append(arrivals_second_half)
    
    # Replace all null values with "unknown"
    df_ber_flights_arrivals = (
        df_ber_flights_arrivals
        .replace({np.nan},'unknown'))
        
    # change date datatype
    df_ber_flights_arrivals["arrival.scheduledTimeLocal"] = pd.to_datetime(df_ber_flights_arrivals["arrival.scheduledTimeLocal"])
    
    # import and update the tables to the SQL
    df_ber_flights_arrivals.to_sql('arrivals_berlin', con=con, if_exists='append', index=False)